# Phase-2: Document Classification using LayoutLMV2 and PyTorch

### Insatalling and Importing necessary dependecies

In [ ]:
!pip install -q pyyaml
!pip install Pillow
!pip install -q transformers
!pip install -q datasets
!pip install transformers[sentencepiece]
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00


In [ ]:
!pip install -q 'git+https://github.com/facebookresearch/detectron2.git'
!pip install -q torchvision tesseract

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:

!sudo apt install tesseract-ocr
!pip install -q pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,760 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
from datasets import Dataset, DatasetDict, load_dataset, Features, Sequence, ClassLabel, Value, Array2D, Array3D, load_from_disk
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
plt.style.use("dark_background")
from typing import Any, List, Tuple

from PIL import Image
from tqdm.notebook import tqdm
from transformers import LayoutLMv2ImageProcessor, LayoutLMv2TokenizerFast, LayoutLMv2Processor, LayoutLMv2ForSequenceClassification, AdamW
import torch

In [ ]:
## Dataset path declaration
path_train = '/content/drive/MyDrive/Datasets/RVL_CDIP_M_600_100/Training'
path_valid = '/content/drive/MyDrive/Datasets/RVL_CDIP_M_600_100/Valid'

In [ ]:
# Creating sorted label list to maintain parity
# labels_list = sorted([label for label in os.listdir(path_train)])     # pass any path as both train and valid folder contains same named and number of folders
labels_list = [label for label in os.listdir(path_train)]
print(labels_list)
print('**********************************************************')
# label to id dictionary format
label2id_dict = {k: v for v,k in enumerate(labels_list)}     # v is label / class name  k is id i.e. 0,1,2...
print(label2id_dict)
print('**********************************************************')
# id to label dictionary format
id2label_dict = {v: k for v,k in enumerate(labels_list)}     # v is label / class name  k is id i.e. 0,1,2...
print(id2label_dict)

['resume', 'handwritten', 'memo', 'scientific_report', 'budget', 'scientific_publication', 'email', 'presentation', 'questionnaire', 'specification', 'invoice', 'letter', 'file_folder', 'form', 'news_article', 'advertisement']
**********************************************************
{'resume': 0, 'handwritten': 1, 'memo': 2, 'scientific_report': 3, 'budget': 4, 'scientific_publication': 5, 'email': 6, 'presentation': 7, 'questionnaire': 8, 'specification': 9, 'invoice': 10, 'letter': 11, 'file_folder': 12, 'form': 13, 'news_article': 14, 'advertisement': 15}
**********************************************************
{0: 'resume', 1: 'handwritten', 2: 'memo', 3: 'scientific_report', 4: 'budget', 5: 'scientific_publication', 6: 'email', 7: 'presentation', 8: 'questionnaire', 9: 'specification', 10: 'invoice', 11: 'letter', 12: 'file_folder', 13: 'form', 14: 'news_article', 15: 'advertisement'}


In [ ]:
len(labels_list)

16

In [ ]:
checkpoint = "microsoft/layoutlmv2-base-uncased"    # create model checkpoin

In [ ]:
## Importing base model of LLMv2
from transformers import LayoutLMv2ImageProcessor, LayoutLMv2TokenizerFast, LayoutLMv2Processor

image_processor = LayoutLMv2ImageProcessor()    ## to apply OCR on the image to get a list of words and normalized bounding boxes
tokenizer = LayoutLMv2TokenizerFast.from_pretrained(checkpoint)    # The words and normalized bounding boxes are then provided which converts them to token-level input_ids, attention_mask, token_type_ids, bbox
processor = LayoutLMv2Processor(image_processor, tokenizer)   # orchestrates the entire pipeline

In [ ]:
## Reload the train and valid dataset dict
processed_dataset_loaded = load_dataset("indra-inc/rvl_cdip_train600_valid100_llmv2_dc")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
processed_dataset_loaded

DatasetDict({
    train: Dataset({
        features: ['image', 'input_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['image', 'input_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels'],
        num_rows: 1600
    })
})

In [ ]:
processed_dataset_loaded['train'].features['labels'].names

['scientific_report',
 'resume',
 'memo',
 'handwritten',
 'letter',
 'budget',
 'invoice',
 'form',
 'presentation',
 'questionnaire',
 'scientific_publication',
 'specification',
 'email',
 'file_folder',
 'news_article',
 'advertisement']

In [ ]:
## Isolate the training and validation dataset
train_dataset = processed_dataset_loaded["train"]
valid_dataset = processed_dataset_loaded["validation"]

In [ ]:
train_dataset

Dataset({
    features: ['image', 'input_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels'],
    num_rows: 9600
})

In [ ]:
valid_dataset

Dataset({
    features: ['image', 'input_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels'],
    num_rows: 1600
})

In [ ]:
from torch.utils.data import DataLoader   # Convert Dataset into PyTorch Dataloader

In [ ]:
# Encoded_dataset.set_format(type = 'torch', device = 'cuda')
train_dataset.set_format(type = 'torch', device = 'cuda' if torch.cuda.is_available() else 'cpu')
valid_dataset.set_format(type = 'torch', device = 'cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# to load the encoded dataset in batches during the training and validation process
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)    # Shuffling dataset for Model Training
valid_dataloader = DataLoader(valid_dataset, batch_size=4)

## Model Execution

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'   # setup device

## define model
model_0 = LayoutLMv2ForSequenceClassification.from_pretrained(checkpoint,
                                                  num_labels=len(labels_list))  # this labels should be label list from the dataframe i.e. all the target column list
## Move model into cuda device
model_0.to(device)

# define optimizer and training parameters
optimizer = AdamW(model_0.parameters(), lr=5e-5)
num_epochs = 10
global_step = 0

# start training
model_0.train()
for epoch in range(num_epochs):
  print(f"Epoch {epoch + 1}/{num_epochs}")

  # Variable initialisation
  running_loss = 0.0    # 'running_loss' is a variable that accumulates the loss value for each batch of data during the training process.
  correct = 0
  # model_0.train()
  for batch in tqdm(train_dataloader):
    # input_ids = batch['input_ids']
    # attention_mask = batch['attention_mask']
    labels_extracted = batch['labels']    # tensor containing the true class labels for the current batch

    # forward pass
    # outputs = model_0(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    pred = model_0(**batch)
    loss = pred.loss     # default loss function i.e CrossEntropyLoss
    running_loss += loss.item()   # loss.item() will extract the value as a Python float, and then add it to the running_loss variable
    # loss.item() returns the value of the loss function for the current batch of data

    # backward pass
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    global_step += 1

    # calculate accuracy
    predictions = torch.argmax(pred.logits, dim=1)    # predictions is a tensor containing the predicted class labels for the current batch

    # pred.logits is a tensor that contains the output of the final layer of the LayoutLMv2 model, which is used to predict the class of the input image
    # torch.argmax() function is used to get the predicted class labels by taking the argmax of the logits along axis 1, which represents the class dimension

    correct += (predictions == labels_extracted).sum().item()   # used to calculate the number of correct predictions

  # print loss and accuracy after each epoch
  avg_loss = running_loss / len(train_dataloader)
  # accuracy = 100 * correct / len(train_dataloader)
  # print(f"Train Loss: {avg_loss:.4f} ::::  Accuracy: {accuracy:.2f}%")
  print(f"Train Loss: {avg_loss:.4f}")


Some weights of LayoutLMv2ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv2-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10


  0%|          | 0/2400 [00:00<?, ?it/s]

Train Loss: 1.3039
Epoch 2/10


  0%|          | 0/2400 [00:00<?, ?it/s]

...

Train Loss: 0.9801
Epoch 9/10


  0%|          | 0/2400 [00:00<?, ?it/s]

Train Loss: 0.8379
Epoch 10/10


  0%|          | 0/2400 [00:00<?, ?it/s]

Train Loss: 0.6893


### Important Code Explanation:


  - In the first line, the torch.argmax() function is used to get the predicted class labels by taking the argmax of the logits along axis 1, which represents the class dimension. pred.logits is a tensor that contains the output of the final layer of the LayoutLMv2 model, which is used to predict the class of the input image. The argmax() function returns the index of the maximum value along the class dimension, which corresponds to the predicted class label.

    - In the second line, (predictions == labels_extracted).sum().item() is used to calculate the number of correct predictions. predictions is a tensor containing the predicted class labels for the current batch, while labels_extracted is a tensor containing the true class labels for the current batch. (predictions == labels_extracted) returns a tensor of the same shape as predictions and labels_extracted, where each element is a boolean value indicating whether the predicted label is equal to the true label. The sum() function is used to add up the number of correct predictions, and the item() function is used to extract this value as a Python integer. Finally, this value is added to the correct variable, which keeps track of the total number of correct predictions made so far during the training process.

  - The running_loss variable has been accumulating the loss values for each batch of data during the training process. In order to get the average loss per sample in the entire training dataset, we divide the running_loss by the total number of samples in the dataset, which is len(test_dataloader.dataset). This gives us the avg_loss.

    - The correct variable has been accumulating the number of correctly classified samples during the training process. In order to get the accuracy of the model on the entire training dataset, we divide the correct by the total number of samples in the dataset, which is again len(test_dataloader.dataset). We then multiply the result by 100 to convert the accuracy to a percentage. This gives us the accuracy. We print both the avg_loss and accuracy to the console after each epoch.

In [36]:
model_0.eval()
with torch.no_grad():
  validation_loss = 0.0
  ## For validation dataset
  for batch in tqdm(valid_dataloader):
        outputs = model_0(**batch)
        loss = outputs.loss

        validation_loss += loss.item()
        predictions = outputs.logits.argmax(-1)

  print("Validation Loss:", validation_loss / len(valid_dataloader))

  0%|          | 0/400 [00:00<?, ?it/s]

Validation Loss: 0.9384442197909811


In [39]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [38]:
## Save processor in drive
processor.save_pretrained("/content/drive/MyDrive/Model_Processor_HF/layoutlmv2_rvlcdip_m")


In [40]:
## Push model to Hub
model_0.push_to_hub("indra-inc/layoutlmv2_rvlcdip_base")

pytorch_model.bin:   0%|          | 0.00/802M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/indra-inc/layoutlmv2_rvlcdip_base/commit/3c3baf143643248b68012ce657a96d40f7b992ca', commit_message='Upload LayoutLMv2ForSequenceClassification', commit_description='', oid='3c3baf143643248b68012ce657a96d40f7b992ca', pr_url=None, pr_revision=None, pr_num=None)

In [41]:
## Reloaded the Model:
model_checkpoint_loaded = "indra-inc/layoutlmv2_rvlcdip_base"

In [42]:
## Inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model = LayoutLMv2ForSequenceClassification.from_pretrained(model_checkpoint_loaded)
loaded_model.to(device)

LayoutLMv2ForSequenceClassification(
  (layoutlmv2): LayoutLMv2Model(
    (embeddings): LayoutLMv2Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (visual): LayoutLMv2VisualBackbone(
      (backbone): FPN(
        (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [44]:
# query = '/content/drive/MyDrive/Datasets/RVL_CDIP_M_600_100/Valid/email/2067311551_2067311552.tif'
# image = Image.open(query).convert("RGB")
# encoded_inputs = processor(image, return_tensors="pt").to(device)
# outputs = loaded_model(**encoded_inputs)
# preds = torch.softmax(outputs.logits, dim=1).tolist()[0]


In [45]:
# pred_labels = {label:pred for label, pred in zip(label2idx.keys(), preds)}
# pred_labels